In [154]:
import json
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors

In [155]:
# -----------------------------
# 1. JSON 파일 읽기
# -----------------------------
with open("../../../measure/ACC/walking_time_data/parks_accessibility_nonhangang.json", "r", encoding="utf-8") as f:
    nonhangang_data = json.load(f)

with open("../../../measure/ACC/walking_time_data/parks_accessibility_hangang.json", "r", encoding="utf-8") as f:
    hangang_data = json.load(f)

In [156]:
# -----------------------------
# 2. GeoDataFrame 불러오기
# -----------------------------
gu = gpd.read_file("../../../map/result/gu.gpkg", layer="gu").to_crs(4326)
parks = gpd.read_file("../../../map/result/parks.gpkg", layer="hangang_parks").to_crs(4326)


In [157]:
# -----------------------------
# 3. 한강 외 공원 → 구별 대표시간 평균 계산
# -----------------------------
gu_times = {}
for park, info in nonhangang_data.items():
    g = info["구"]
    time = info["최종대표시간"]

    # 구가 리스트 형태일 수도 있음 (북한산 등)
    if isinstance(g, list):
        for gg in g:
            gu_times.setdefault(gg, []).append(time)
    else:
        gu_times.setdefault(g, []).append(time)

gu_avg = {g: sum(times) / len(times) for g, times in gu_times.items()}

gu_df = pd.DataFrame([{"자치구": g, "최종대표시간": t} for g, t in gu_avg.items()])

# merge (자치구 컬럼은 drop해서 충돌 방지)
gu = gu.merge(
    gu_df,
    left_on="SIGUNGU_NM",
    right_on="자치구",
    how="left"
).drop(columns=["자치구"])

In [158]:
gu.head()

,BASE_DATE,SIGUNGU_NM,SIGUNGU_CD,geometry,최종대표시간
0,20240630,동대문구,11060,"MULTIPOLYGON (((127.06937 37.60828, 127.06938 ...",9.250
1,20240630,중랑구,11070,"MULTIPOLYGON (((127.11131 37.62069, 127.11193 ...",4.625
2,20240630,성북구,11080,"MULTIPOLYGON (((126.98396 37.63644, 126.9842 3...",8.500
3,20240630,강북구,11090,"MULTIPOLYGON (((127.00459 37.68507, 127.00553 ...",11.500
4,20240630,도봉구,11100,"MULTIPOLYGON (((127.01983 37.70102, 127.02215 ...",6.250


In [159]:
# -----------------------------
# 4. 한강공원 → 개별 공원 대표시간
# -----------------------------
park_df = pd.DataFrame([
    {"공원명": info["공원명"], "최종대표시간": info["최종대표시간"]}
    for _, info in hangang_data.items()
])

# 영어 park_name → 한국어 변환 매핑
park_name_map = {
    "Banpo": "반포한강공원",
    "Gangseo": "강서한강공원",
    "Gwangnaru": "광나루한강공원",
    "Ichon": "이촌한강공원",
    "Jamsil": "잠실한강공원",
    "Jamwon": "잠원한강공원",
    "Mangwon": "망원한강공원",
    "Nanji": "난지한강공원",
    "Ttukseom": "뚝섬한강공원",
    "Yanghwa": "양화한강공원",
    "Yeouido": "여의도한강공원"
}

# 한글로 변환
parks["park_name"] = parks["park_name"].map(park_name_map)

# merge (공원명 컬럼은 drop해서 충돌 방지)
parks = parks.merge(
    park_df,
    left_on="park_name",
    right_on="공원명",
    how="left"
).drop(columns=["공원명"])

In [160]:
# -----------------------------
# 6. folium 지도 생성
# -----------------------------
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11, tiles="CartoDB positron")

# colormap 기준: 구 + 한강공원 전체 최종대표시간
all_min = min(gu["최종대표시간"].min(), parks["최종대표시간"].min())
all_max = max(gu["최종대표시간"].max(), parks["최종대표시간"].max())

cmap = plt.cm.Purples_r  # 둘 다 같은 팔레트 사용
norm = colors.Normalize(vmin=all_min, vmax=all_max)

# 자치구 레이어
folium.GeoJson(
    gu,
    style_function=lambda f: {
        "fillColor": "#eeeeee" if pd.isna(f["properties"]["최종대표시간"]) else 
                     colors.rgb2hex(cmap(norm(f["properties"]["최종대표시간"]))),
        "color": "#666",
        "weight": 1,
        "fillOpacity": 0.8
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["SIGUNGU_NM", "최종대표시간"],
        aliases=["자치구", "대표시간(분)"]
    )
).add_to(m)

# 한강공원 레이어
folium.GeoJson(
    parks,
    name="한강공원",
    style_function=lambda f: {
        "fillColor": "#cccccc" if pd.isna(f["properties"]["최종대표시간"]) else 
                     colors.rgb2hex(cmap(norm(f["properties"]["최종대표시간"]))),
        "color": "#666",
        "weight": 1.2,
        "fillOpacity": 0.8
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["park_name", "최종대표시간"],
        aliases=["공원", "대표시간(분)"]
    )
).add_to(m)

# 레이어 컨트롤
folium.LayerControl(collapsed=False).add_to(m)


In [161]:
# 저장
m.save("../../result/ACC/walking_time_visualization.html")
print("walking_time_visualization.html 저장 완료")

walking_time_visualization.html 저장 완료
